<a href="https://colab.research.google.com/github/takadeanur/whisper/blob/main/%E3%80%90%E2%98%85%E2%98%8512fps%E3%81%8B%E3%82%8960fps%E5%AE%9F%E8%A1%8C%E5%8F%AF%E8%83%BDAPI%E7%89%88%E3%80%91%E9%80%9A%E5%B8%B8%E3%81%AE%E3%82%A2%E3%83%83%E3%83%97%E3%82%B9%E3%82%B1%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# 0. 初期準備と依存関係のインポート
# ===============================
import json
import requests
import time
import subprocess
import threading
import os
import shutil
import re
import datetime
import random
from ftplib import FTP
from google.colab import userdata
from typing import List, Tuple, Optional

# Colabランタイム切断用モジュールをインポート
try:
    from google.colab import runtime
except ImportError:
    class MockRuntime:
        def unassign(self):
            print("🚨 警告: Colab環境外です。runtime.unassign() は実行されません。")
    runtime = MockRuntime()

# ===============================
# 1. 環境準備
# ===============================
print("--- 1. 環境準備 ---")
# aria2のインストール
!apt -y install -qq aria2
# ffmpeg (ffprobeを含む) は、前のステップでインストール済み

# ===============================
# 2. ComfyUI 本体の取得
# ===============================
print("\n--- 2. ComfyUI 本体の取得 ---")
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!git reset --hard c60dc4177c16d50da025bda4ac7fd513bb86e699
!pip install -r requirements.txt
!pip install sageattention==1.0.6

# ===============================
# 3. カスタムノードの追加
# ===============================
print("\n--- 3. カスタムノードの追加 ---")
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git
!git clone https://github.com/M1kep/ComfyLiterals
!git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git

# ---------------- ComfyUI-Frame-Interpolationの設定 ---------------------
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/Fannovel16/ComfyUI-Frame-Interpolation.git
%cd ComfyUI-Frame-Interpolation
!python install.py
%cd /content/ComfyUI/custom_nodes
# ---------------- ComfyUI-Frame-Interpolationの設定完了 ---------------------

# ---------------- rgthree-comfyの設定 ---------------------
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/rgthree/rgthree-comfy
%cd rgthree-comfy
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes
# ---------------- rgthree-comfyの設定完了 ---------------------

# =========================================================================
# 5. FTP連携とAPI実行用のPythonコード (定数・設定)
# =========================================================================

# --- 5-1. ファイルパスと設定（FTP関連） ---
PORT = 8188
COMFYUI_PORT = PORT
JSON_FILE_NAME = "single_video_workflow.json"
PROMPT_FILE_PATH = f"/content/{JSON_FILE_NAME}"
COMFYUI_SERVER_URL = f"http://127.0.0.1:{COMFYUI_PORT}"

# ★★★ 動画生成の強制再実行フラグ (ここを切り替えます) ★★★
FORCE_RECREATE = False
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
MAX_VIDEOS_TO_PROCESS = 35

# シークレット情報の取得
print("\n--- 5-2. FTP接続情報の取得とファイル名設定 ---")
try:
    FTP_HOST = userdata.get('ftp_host')
    FTP_USER = userdata.get('ftp_user')
    FTP_PASS = userdata.get('ftp_pass')
except:
    FTP_HOST = "your_ftp_host"
    FTP_USER = "your_ftp_user"
    FTP_PASS = "your_ftp_pass"


if not (FTP_HOST and FTP_USER and FTP_PASS) or FTP_HOST == "your_ftp_host":
    print("🚨 警告: FTP接続情報が取得できないか、テスト値のままです。FTP機能は動作しません。")
else:
    print("✅ FTP接続情報の取得に成功しました。")

# --- 入力画像のベースディレクトリ設定 ---
FTP_BASE_DIR = "/jitantech.com/public_html/py-project/youtube-news-project"
INPUT_BASE_DIR_NAME = "downloaded_images_by_channel_id_rank"
INPUT_BASE_FTP_FOLDER = f"{FTP_BASE_DIR}/{INPUT_BASE_DIR_NAME}/"

# ワークフローの LoadVideo ノードが参照する固定ファイル名とパス
# RIFE VFI の入力ファイル名と、連番出力のベース名

# --- 🚨 修正後の固定ファイル名設定 ---
TARGET_VIDEO_FILENAME = "rap_001_720px_8x.mp4" # ★ ターゲット入力ファイル名 (FTP上)
# ★ 最終出力ファイル名を gyang_001_720px_8x_72fps.mp4 に固定
FINAL_OUTPUT_FILENAME = "rap_001_720px_8x_72fps.mp4"
# ★ VideoCombineノードの出力接頭辞を修正
OUTPUT_VIDEO_BASE_NAME = "rap_001_720px_8x_72fps"

# ★ ワークフローの LoadVideo ノードが参照する ComfyUI input フォルダ内のファイル名をターゲットのファイル名に合わせる
COMFYUI_INPUT_FILENAME = TARGET_VIDEO_FILENAME
# -----------------------------------

INPUT_IMAGE_LOCAL_PATH = f"/content/ComfyUI/input/{COMFYUI_INPUT_FILENAME}"
LOAD_VIDEO_NODE_ID = "10"
VIDEO_COMBINE_NODE_ID = "13"
COMBINE_PREFIX_KEY = "filename_prefix"


print(f"⚙️ 処理対象の画像ベースフォルダ: {INPUT_BASE_FTP_FOLDER}")
print(f"⚙️ ComfyUI入力ファイル名: {COMFYUI_INPUT_FILENAME} (←修正済み)")
print(f"⚙️ 最終出力ファイル名: {FINAL_OUTPUT_FILENAME} (←修正済み)")


# --- 5-3. ワークフローJSONをPythonオブジェクトとして定義 (RAM対策解除適用) ---
# 🚨 10番ノードの "video" 入力を COMFYUI_INPUT_FILENAME に合わせる (コード上の値と定数値を一致させる)
workflow = {
    "10": {
        "inputs": {
            "video": COMFYUI_INPUT_FILENAME, # ★ 修正: 定数値と一致
            "force_rate": 0,
            "custom_width": 3840,
            "custom_height": 2160,
            "frame_load_cap": 0,
            "skip_first_frames": 1,
            "select_every_nth": 1,
            "format": "AnimateDiff"
        },
        "class_type": "VHS_LoadVideo",
        "_meta": {
            "title": "Load Video (Upload) 🎥🅥🅗🅢"
        }
    },
    "13": {
        "inputs": {
            "frame_rate": [
                "34",
                1
            ],
            "loop_count": 0,
            "filename_prefix": "Upscaled",
            "format": "video/h264-mp4",
            "pix_fmt": "yuv420p",
            "crf": 10,
            "save_metadata": True,
            "trim_to_audio": False,
            "pingpong": False,
            "save_output": True,
            "images": [
                "27",
                0
            ],
            "audio": [
                "10",
                2
            ]
        },
        "class_type": "VHS_VideoCombine",
        "_meta": {
            "title": "Video Combine 🎥🅥🅗🅢"
        }
    },
    "27": {
        "inputs": {
            "ckpt_name": "rife47.pth",
            "clear_cache_after_n_frames": 10,
            "multiplier": [
                "35",
                0
            ],
            "fast_mode": True,
            "ensemble": True,
            "scale_factor": 1,
            "frames": [
                "10",
                0
            ]
        },
        "class_type": "RIFE VFI",
        "_meta": {
            "title": "RIFE VFI (recommend rife47 and rife49)"
        }
    },
    "34": {
        "inputs": {
            "expression": "a * b",
            "a": [
                "40",
                0
            ],
            "b": [
                "35",
                0
            ]
        },
        "class_type": "MathExpression|pysssss",
        "_meta": {
            "title": "Math Expression 🐍"
        }
    },
    "35": {
        "inputs": {
            "Number": "5"
        },
        "class_type": "Int",
        "_meta": {
            "title": "FPS Multipler"
        }
    },
    "40": {
        "inputs": {
            "video_info": [
                "10",
                3
            ]
        },
        "class_type": "VHS_VideoInfo",
        "_meta": {
            "title": "Video Info 🎥🅥🅗🅢"
        }
    }
}

# --- JSON ワークフローを事前に書き出す ---
%cd /content
with open(JSON_FILE_NAME, 'w') as f:
    json.dump(workflow, f, indent=2)
print(f"✅ ワークフローJSONを {PROMPT_FILE_PATH} に書き出しました。")
%cd /content/ComfyUI


# =======================================================
# 6. 動画リスト取得関数
# =======================================================
def get_unprocessed_image_list() -> List[Tuple[str, str]]:
    """
    FTPベースディレクトリ以下の全サブフォルダを検索し、
    最終結合ファイル ('gyang_001_720px_8x_72fps.mp4') が存在しない動画をリストアップします。
    """
    FINAL_OUTPUT_PATTERN = FINAL_OUTPUT_FILENAME

    if FORCE_RECREATE:
        print(f"\n⚙️ **強制再生成モード** が有効です。全フォルダの '{TARGET_VIDEO_FILENAME}' を処理対象とします。")
    else:
        print(f"\n⚙️ **未生成チェックモード** が有効です。最終ファイル '{FINAL_OUTPUT_PATTERN}' の有無を全フォルダでチェックします。")

    print(f"FTP サーバー ({INPUT_BASE_FTP_FOLDER}) から対象動画を検索します...")
    unprocessed_videos = []
    found_any_target = False

    try:
        with FTP(FTP_HOST) as ftp:
            ftp.login(user=FTP_USER, passwd=FTP_PASS)

            sub_dir_full_paths = []
            for item in ftp.nlst(INPUT_BASE_FTP_FOLDER):
                # フォルダのみを対象とするための簡易チェック
                if not os.path.splitext(item)[1]:
                    sub_dir_full_paths.append(item)

            processed_sub_dirs = sorted(sub_dir_full_paths)

            if not processed_sub_dirs:
                print("⚠️ 処理すべきサブフォルダ（チャンネルIDフォルダ）が見つかりませんでした。")
                return []

            for folder_path in processed_sub_dirs:
                if not folder_path.endswith('/'): folder_path += '/'
                log_folder_name = os.path.basename(os.path.normpath(folder_path))

                source_filenames = []
                try:
                    # フォルダ内のファイルリストを取得
                    source_filenames = [os.path.basename(f) for f in ftp.nlst(folder_path)]
                except:
                    continue

                if TARGET_VIDEO_FILENAME in source_filenames:
                    found_any_target = True
                    full_ftp_path = os.path.join(folder_path, TARGET_VIDEO_FILENAME)

                    if FORCE_RECREATE:
                        unprocessed_videos.append((full_ftp_path, TARGET_VIDEO_FILENAME))
                        print(f"➡️ 強制処理対象: フォルダ: {log_folder_name}")
                    elif FINAL_OUTPUT_PATTERN not in source_filenames:
                        unprocessed_videos.append((full_ftp_path, TARGET_VIDEO_FILENAME))
                        print(f"➡️ 未生成: フォルダ: {log_folder_name}")
                    else:
                        print(f"✅ 存在確認OK: フォルダ: {log_folder_name}, ファイル: {FINAL_OUTPUT_PATTERN} が存在するためスキップします。")

                if len(unprocessed_videos) >= MAX_VIDEOS_TO_PROCESS:
                    print(f"⚠️ 処理対象の検出が最大処理件数 {MAX_VIDEOS_TO_PROCESS} 件に達しました。")
                    break

            if not found_any_target:
                return []
            if not unprocessed_videos:
                return []

            print(f"✅ 合計 {len(unprocessed_videos)} 件の動画ファイルを処理対象として見つけました。")

            if len(unprocessed_videos) > MAX_VIDEOS_TO_PROCESS:
                unprocessed_videos = unprocessed_videos[:MAX_VIDEOS_TO_PROCESS]

            return unprocessed_videos

    except Exception as e:
        raise Exception(f"🚨 FTP (動画リスト取得) 中に致命的なエラーが発生しました: {e}")


# =======================================================
# 7. FTPからの単一動画ダウンロード
# =======================================================
def download_image_to_comfyui_input(ftp_image_full_path, local_input_path, verbose=True):
    """ 指定されたFTP動画をダウンロードし、ComfyUIの固定入力パスに保存します。"""
    if verbose:
        print(f"⚙️ 入力動画 {ftp_image_full_path} を {local_input_path} にダウンロード中...")
    try:
        with FTP(FTP_HOST) as ftp:
            ftp.login(user=FTP_USER, passwd=FTP_PASS)
            if os.path.exists(local_input_path):
                os.remove(local_input_path)
            with open(local_input_path, 'wb') as f:
                ftp.retrbinary(f'RETR {ftp_image_full_path}', f.write)
        if not os.path.exists(local_input_path):
            raise FileNotFoundError(f"FTPダウンロード後、ファイル {local_input_path} が見つかりません。")
        if verbose:
            print(f"✅ ダウンロード完了。ComfyUIの入力ファイル名: {os.path.basename(local_input_path)}")
        return True
    except Exception as e:
        if verbose:
            print(f"🚨 動画ダウンロード中にエラーが発生しました: {e}")
        return False


# =======================================================
# 8. FTPへの動画アップロード
# =======================================================
def upload_video_to_ftp(local_video_path, ftp_source_image_full_path, generated_video_name):
    """
    生成された動画をFTPサーバー上の、元の動画と同じフォルダにアップロードします。
    """
    if not local_video_path or not os.path.exists(local_video_path):
        print(f"🚨 エラー: 生成動画ファイル {local_video_path} が見つかりません。アップロードをスキップします。")
        return
    ftp_target_dir = os.path.dirname(ftp_source_image_full_path) + "/"
    remote_path = os.path.join(ftp_target_dir, generated_video_name)
    print(f"\nFTP サーバー ({ftp_target_dir}) へ生成動画 ({generated_video_name}) をアップロードします...")
    try:
        with FTP(FTP_HOST) as ftp:
            ftp.login(user=FTP_USER, passwd=FTP_PASS)
            with open(local_video_path, 'rb') as file:
                ftp.storbinary(f'STOR {remote_path}', file)
            print(f"✅ 生成動画 {generated_video_name} を元の動画と同じディレクトリにアップロードしました。")
    except Exception as e:
        print(f"🚨 FTP (動画アップロード) 中にエラーが発生しました: {e}")

# =======================================================
# 9. 動画情報取得関数 (ffprobe)
# =======================================================
def get_original_fps(local_video_path: str) -> Optional[float]:
    """ ffprobe を使用して元の動画のフレームレートを取得します。"""
    try:
        cmd = [
            "ffprobe",
            "-v", "error",
            "-select_streams", "v:0",
            "-show_entries", "stream=avg_frame_rate",
            "-of", "csv=p=0",
            local_video_path
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        rate_str = result.stdout.strip()
        if '/' in rate_str:
            num, den = map(int, rate_str.split('/'))
            return num / den
        else:
            return float(rate_str)
    except Exception as e:
        print(f"🚨 ffprobeでFPS取得中にエラーが発生しました: {e}")
        return None

# =======================================================
# 10. 単一動画処理実行関数
# =======================================================
def process_single_video(local_video_path, ftp_source_path, prompt_file_path, server_url, base_output_name, final_output_ftp_name):
    """
    動画全体を一度にComfyUI APIで処理し、元の動画から音声を抽出し結合します。
    """
    print(f"\n--- 動画全体を処理開始 ---")

    # 1. ワークフローをロードし、ファイル名を更新
    with open(prompt_file_path, 'r') as f:
        prompt = json.load(f)

    # ワークフローにはフレーム数設定がないため、ファイル名のみ更新
    # LoadVideoノードのファイル名は、JSONロード時に定義したCOMFYUI_INPUT_FILENAMEになっているため、ここでは変更不要。
    prompt[VIDEO_COMBINE_NODE_ID]["inputs"]["filename_prefix"] = base_output_name

    # 2. ワークフロー実行のリクエスト
    client_id = "colab_client_" + str(os.getpid())
    prompt_url = f"{server_url}/prompt"
    payload = {"prompt": prompt, "client_id": client_id}

    response = None
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.post(prompt_url, json=payload, timeout=5)
            response.raise_for_status()
            break
        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                print(f"警告: API接続エラー (試行 {attempt + 1}/{max_retries})。再試行します: {e}")
                time.sleep(2)
            else:
                print(f"🚨 API接続エラーが解決しませんでした: {e}")
                return False

    # 3. 実行完了のポーリング
    prompt_id = response.json().get('prompt_id')
    history_url = f"{server_url}/history/{prompt_id}"
    start_poll_time = time.time()
    # チャンクより長めに待機時間を設定
    max_poll_time = 7200

    while time.time() - start_poll_time < max_poll_time:
        time.sleep(5)
        try:
            history_response = requests.get(history_url)
            history_response.raise_for_status()
            if prompt_id in history_response.json():
                print("動画処理が完了しました。")
                break
        except requests.exceptions.RequestException:
            continue
    else:
        print(f"🚨 エラー: 動画処理が {max_poll_time} 秒以内に完了しませんでした。")
        return False

    # 4. 生成された動画ファイルを特定
    generated_ok = False
    latest_video_path = None
    comfy_output_dirs = ["/content/ComfyUI/output/", "/content/ComfyUI/output/video/"]

    for check_dir in comfy_output_dirs:
        if not os.path.exists(check_dir): continue
        expected_prefix = base_output_name
        for file in os.listdir(check_dir):
            if file.startswith(expected_prefix) and file.endswith(".mp4"):
                file_path = os.path.join(check_dir, file)
                # ファイル名が完全一致したもの (VideoCombineの出力) を探し、最新のものを選択
                # ※ VideoCombineは通常、`接頭辞_00001_.mp4` のような形で出力するため、ここでは接頭辞でチェックする
                if file.startswith(expected_prefix) and (not latest_video_path or os.path.getmtime(file_path) > os.path.getmtime(latest_video_path)):
                    latest_video_path = file_path

    if not latest_video_path:
        print("🚨 致命的エラー: 生成動画ファイルが見つかりませんでした。")
        return False

    # 5. 生成されたファイルを最終出力名にリネームしてルートに移動
    VFI_VIDEO_LOCAL_PATH = latest_video_path
    FINAL_OUTPUT_LOCAL_PATH = f"/content/{final_output_ftp_name}"

    try:
        shutil.move(VFI_VIDEO_LOCAL_PATH, FINAL_OUTPUT_LOCAL_PATH)
        print(f"✅ 生成動画を {FINAL_OUTPUT_LOCAL_PATH} に保存しました。")
    except Exception as e:
        print(f"🚨 ファイル移動/リネーム中にエラーが発生しました: {e}")
        return False


    # 6. FTPにアップロード
    upload_video_to_ftp(FINAL_OUTPUT_LOCAL_PATH, ftp_source_path, final_output_ftp_name)

    # 7. 最終クリーンアップ (ローカルの生成ファイル)
    try:
        os.remove(FINAL_OUTPUT_LOCAL_PATH)
        print(f"🗑️ 最終結合ファイル {FINAL_OUTPUT_LOCAL_PATH} をクリーンアップしました。")
    except Exception as e:
        print(f"⚠️ 最終クリーンアップに失敗: {e}")

    return True


# ===============================
# 11. ComfyUIサーバー起動 (RAM対策: --lowvram を削除)
# ===============================
def start_comfyui_server(port):
    print(f"Starting ComfyUI server in API mode on port {port}...")
    # ★ 修正: --lowvram を削除し、高速化を試みる
    comfyui_process = subprocess.Popen(
        f"python main.py --listen --port {port}",
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        cwd="/content/ComfyUI"
    )
    return comfyui_process

def wait_for_server(process, port):
    server_url = f"http://127.0.0.1:{port}"
    start_time = time.time()
    max_wait = 60
    print("ComfyUIサーバー起動を待機中...")
    while time.time() - start_time < max_wait:
        try:
            response = requests.get(f"{server_url}/queue", timeout=1)
            if response.status_code == 200:
                print("✅ ComfyUIサーバーの起動を確認しました。")
                return True
        except requests.exceptions.RequestException:
            pass
        line = process.stdout.readline()
        if line:
            print(f"ComfyUI Log: {line.strip()}")
            if "To see the GUI go to:" in line or "Unet loaded" in line or "Running on" in line:
                print("サーバーからのログにより、起動確認をしました。")
                return True
        time.sleep(1)
    print("🚨 エラー: ComfyUIサーバーが60秒以内に起動しませんでした。")
    return False


# ===============================
# 12. 全体実行フロー
# ===============================
if __name__ == "__main__":

    unprocessed_list = []
    comfyui_process = None

    # 1. FTPから未処理動画のリストを取得
    try:
        unprocessed_list = get_unprocessed_image_list()
    except Exception as e:
        print(f"処理を中断します。エラー: {e}")
        exit()

    if not unprocessed_list:
        print("処理対象の動画ファイルが見つからなかったか、既にアップスケール済みです。処理を終了します。")
        exit()

    # 2. ComfyUIサーバーの起動
    print("\n--- ComfyUIサーバー起動 (API専用) ---")
    comfyui_process = start_comfyui_server(COMFYUI_PORT)

    # 3. サーバー起動確認
    if not wait_for_server(comfyui_process, COMFYUI_PORT):
        if comfyui_process:
            comfyui_process.terminate()
        exit()

    # 4. API クライアントを実行して動画を生成 (リストをループ処理)
    processed_count = 0
    total_to_process = len(unprocessed_list)

    print(f"\n--- 合計 {total_to_process} 件の動画に対するループ処理を開始します (最大 {MAX_VIDEOS_TO_PROCESS} 件) ---")

    # 🚨 ComfyUIの入力/出力ファイル名は固定値を使用
    final_output_name = FINAL_OUTPUT_FILENAME
    output_base_name = OUTPUT_VIDEO_BASE_NAME

    for i, (input_image_ftp_full_path, input_image_filename) in enumerate(unprocessed_list):

        if processed_count >= MAX_VIDEOS_TO_PROCESS:
            print(f"\n✅ 最大処理件数 {MAX_VIDEOS_TO_PROCESS} 件に達したため、ループを中断します。")
            break

        # 🚨 FTP上のファイル名と TARGET_VIDEO_FILENAME が異なる場合はエラーになります。
        # 🚨 このシステムは、すべてのターゲットフォルダに TARGET_VIDEO_FILENAME があることを前提とします。
        if input_image_filename != TARGET_VIDEO_FILENAME:
            print(f"⚠️ 警告: FTP上のファイル名 ({input_image_filename}) がターゲットファイル名 ({TARGET_VIDEO_FILENAME}) と異なります。この処理フローではスキップします。")
            continue

        folder_path = os.path.dirname(input_image_ftp_full_path)

        print(f"\n\n========================================================")
        print(f"== 処理開始: {i + 1}/{total_to_process} - フォルダ: {os.path.basename(os.path.normpath(folder_path))} ==")
        print(f"➡️ 入力ファイル名: {input_image_filename}")
        print(f"➡️ 最終出力ファイル名: {final_output_name}")
        print(f"========================================================")


        # 4a. 入力動画をダウンロード (INPUT_IMAGE_LOCAL_PATH = /content/ComfyUI/input/gyang_001_720px_8x.mp4 になる)
        if not download_image_to_comfyui_input(input_image_ftp_full_path, INPUT_IMAGE_LOCAL_PATH, verbose=True):
            continue

        # 4b. 単一処理を実行
        if process_single_video(
            local_video_path=INPUT_IMAGE_LOCAL_PATH,
            ftp_source_path=input_image_ftp_full_path,
            prompt_file_path=PROMPT_FILE_PATH,
            server_url=COMFYUI_SERVER_URL,
            base_output_name=output_base_name,
            final_output_ftp_name=final_output_name
        ):
            # 処理が成功した場合のみカウントアップ
            processed_count += 1

        # 4c. クリーンアップ (ComfyUI入力)
        if os.path.exists(INPUT_IMAGE_LOCAL_PATH):
            os.remove(INPUT_IMAGE_LOCAL_PATH)
            print(f"🗑️ {INPUT_IMAGE_LOCAL_PATH} (ComfyUI入力) を削除しました。")

        print(f"🔄 現在の処理完了動画数: {processed_count}/{total_to_process} (フォルダ: {os.path.basename(os.path.normpath(folder_path))})")


    # 5. 処理終了後、プロセスを終了
    if comfyui_process:
        comfyui_process.terminate()
        print("\nComfyUI プロセスを終了しました。")

    # 6. 最終クリーンアップ
    print("\n--- 最終クリーンアップ ---")
    if os.path.exists(PROMPT_FILE_PATH):
        os.remove(PROMPT_FILE_PATH)
        print(f"🗑️ {PROMPT_FILE_PATH} を削除しました。")


    print("\n--- 全ての処理が完了しました ---")

    # 7. ランタイム自動終了ロジック
    if processed_count >= MAX_VIDEOS_TO_PROCESS:
        print("\n🚨 処理件数上限に達しました。ランタイムを自動的に切断します。")
        runtime.unassign()

--- 1. 環境準備 ---
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up a